In [ ]:
# Cell 1: Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt


### 📚 Top 50 SIFT Correspondences Between Book Image and Video Frame

This cell defines the function `get_top_50_correspondences`, which identifies and visualizes the top 50 SIFT feature matches between a given book cover image and the first frame of a video. Here's what it does step-by-step:

1. **Loads the Book Image and First Video Frame**  
   - Converts both to grayscale for feature extraction.

2. **Extracts SIFT Keypoints and Descriptors**  
   - Uses the Scale-Invariant Feature Transform (SIFT) algorithm to detect and describe keypoints in both images.

3. **Matches Features Using BFMatcher with k-Nearest Neighbors**  
   - Finds the two closest matches for each descriptor using brute-force matching.

4. **Applies Lowe's Ratio Test**  
   - Filters out poor matches by keeping only those that pass the 0.75 ratio threshold.

5. **Selects the Top 50 Best Matches**  
   - Sorts the good matches by distance and selects the 50 most accurate ones.

6. **Visualizes the Matches**  
   - Draws the top 50 matches between the book image and the video frame.

This is a crucial step for establishing correspondences that can be used later for homography computation and AR overlays.


In [ ]:
# Cell 2: Function to get top 50 SIFT matches
def get_top_50_correspondences(book_img_path, book_video_path):
    # Load book image
    img1 = cv2.imread(book_img_path)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    # Load first frame from video
    cap = cv2.VideoCapture(book_video_path)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise RuntimeError("Failed to read the first frame from video.")
    img2 = frame
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # SIFT + BFMatcher
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1_gray, None)
    kp2, des2 = sift.detectAndCompute(img2_gray, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Lowe's Ratio Test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    # Select top 50 matches
    top_50_matches = sorted(good, key=lambda x: x.distance)[:50]

    # Visualization
    match_img = cv2.drawMatches(img1, kp1, img2, kp2, top_50_matches, None, flags=2)

    plt.figure(figsize=(14, 7))
    plt.imshow(cv2.cvtColor(match_img, cv2.COLOR_BGR2RGB))
    plt.title("Top 50 Correspondences between Book Image and First Video Frame")
    plt.axis('off')
    plt.show()

    return top_50_matches, kp1, kp2, img1, img2


In [ ]:
# Cell 3: Run matching
book_image_path = r"C:\Users\User\PycharmProjects\pythonProject5\cv_cover.jpg"
book_video_path = r"C:\Users\User\PycharmProjects\pythonProject5\book.mov"

top_matches, kp1, kp2, img1, img2 = get_top_50_correspondences(book_image_path, book_video_path)


### 🔁 Computing the Homography Matrix from Point Correspondences

This function `compute_homography_matrix` calculates the **homography transformation matrix (H)** from two sets of corresponding 2D points using the **Direct Linear Transform (DLT)** algorithm.

#### What the Function Does:
1. **Checks for Valid Input**  
   - Ensures the input point sets `pts1` and `pts2` have the same shape and contain at least 4 correspondences, which is the minimum needed for computing a homography.

2. **Constructs the Linear System (Matrix A)**  
   - For each correspondence between a point in the source image `(x, y)` and the destination image `(x', y')`, two rows are added to matrix A based on the homography equation.

3. **Solves Using Singular Value Decomposition (SVD)**  
   - Uses SVD to solve the homogeneous system `Ah = 0`, where `h` contains the flattened elements of the homography matrix.
   - The solution is taken as the last row of `V^T` (i.e., the eigenvector corresponding to the smallest singular value).

4. **Reshapes and Normalizes**  
   - Reshapes the solution vector into a 3×3 matrix `H`.
   - Normalizes the matrix so that `H[2, 2] = 1`.

#### Why It's Important:
This homography matrix allows us to **warp one image to align with another**, which is essential for tasks like:
- Image stitching
- Augmented reality overlays
- Perspective correction


In [ ]:
# Cell 4: Homography function
def compute_homography_matrix(pts1, pts2):
    assert pts1.shape == pts2.shape, "Input point arrays must have same shape"
    assert pts1.shape[0] >= 4, "At least 4 point correspondences required"

    n = pts1.shape[0]
    A = []

    for i in range(n):
        x, y = pts1[i][0], pts1[i][1]
        x_prime, y_prime = pts2[i][0], pts2[i][1]

        A.append([-x, -y, -1, 0, 0, 0, x * x_prime, y * x_prime, x_prime])
        A.append([0, 0, 0, -x, -y, -1, x * y_prime, y * y_prime, y_prime])

    A = np.asarray(A)
    _, _, Vt = np.linalg.svd(A)
    H = Vt[-1].reshape((3, 3))

    return H / H[2, 2]


### 🔄 Warp Points Using a Homography Matrix (Optional Verification)

The `warp_points` function applies a **homography transformation** to a set of 2D points.

#### Function Purpose:
This is a utility function used to **verify the effect of the computed homography matrix** by transforming a set of source points into the new perspective (destination space).

#### How It Works:
1. **Convert Points to Homogeneous Coordinates**  
   - Adds a third coordinate (1) to each 2D point to make them homogeneous:  
     \[(x, y) → (x, y, 1)\]

2. **Apply the Homography Matrix**  
   - Multiplies each point with the homography matrix \(H\) to get the transformed coordinates.

3. **Normalize the Result**  
   - Converts from homogeneous back to Cartesian coordinates by dividing by the third (homogeneous) coordinate.

4. **Returns the Transformed Points**  
   - The result is a new array of shape `(num_points, 2)` containing the warped \(x'\) and \(y'\) coordinates.

#### When to Use:
This function is helpful for:
- Verifying that the homography correctly maps points.
- Debugging by checking where keypoints land after transformation.


In [ ]:
# Cell 5: Warp point verification (optional)
def warp_points(H, pts):
    num_pts = pts.shape[0]
    pts_homogeneous = np.hstack((pts, np.ones((num_pts, 1))))
    warped = H @ pts_homogeneous.T
    warped /= warped[2]
    return warped[:2].T


In [ ]:
# Cell 6: Compute homography from previous matches
pts1 = np.float32([kp1[m.queryIdx].pt for m in top_matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in top_matches])

H = compute_homography_matrix(pts1, pts2)
print("Homography matrix H:\n", H)

# Optional: verify error
warped_pts1 = warp_points(H, pts1)
error = np.linalg.norm(warped_pts1 - pts2, axis=1).mean()
print("Mean projection error: {:.4f}".format(error))


### 🗺️ Mapping Book Corners to Video Frame & Drawing Bounding Box

#### 📐 `get_book_corners_in_video(book_img_path, H)`
This function maps the four corners of the book image into the video frame using the **homography matrix** \(H\).

##### What It Does:
1. **Load Book Image**  
   Reads the image to determine its dimensions.

2. **Define Corners in Image Coordinates**  
   Orders corners in this order: top-left, top-right, bottom-right, bottom-left.

3. **Convert to Homogeneous Coordinates**  
   Adds a third dimension (1) to allow for matrix multiplication with \(H\).

4. **Apply Homography**  
   Transforms the book corners to video frame coordinates.

5. **Return**  
   - Original corner coordinates.  
   - Warped (mapped) corner coordinates.

---

#### 🖍️ `draw_book_corners_on_frame(frame, corners, color=(0, 255, 0))`
This function draws the **projected quadrilateral** (the book's corners) on a given video frame.

##### What It Does:
- Converts the float coordinates to integers.
- Uses `cv2.polylines` to draw the 4-corner polygon (typically a rectangle).
- The result is a frame with the book's outline drawn on it.

##### Usage Example:
Useful for visual debugging or creating visual overlays showing where the book is detected in each frame.


In [ ]:
# Cell 7: Function to compute mapped corners
def get_book_corners_in_video(book_img_path, H):
    book_img = cv2.imread(book_img_path)
    h, w = book_img.shape[:2]
    book_corners = np.array([
        [0, 0],
        [w - 1, 0],
        [w - 1, h - 1],
        [0, h - 1]
    ], dtype=np.float32)

    ones = np.ones((4, 1))
    book_corners_hom = np.hstack([book_corners, ones])

    mapped = H @ book_corners_hom.T
    mapped /= mapped[2]
    return book_corners, mapped[:2].T


In [ ]:
# Cell 8: Draw mapped corners on frame
def draw_book_corners_on_frame(frame, corners, color=(0, 255, 0)):
    corners = np.int32(corners)
    frame_with_box = frame.copy()
    cv2.polylines(frame_with_box, [corners], isClosed=True, color=color, thickness=2)
    return frame_with_box


In [ ]:
# Cell 9: Apply mapping and draw
original_corners, mapped_corners = get_book_corners_in_video(book_image_path, H)
frame_with_box = draw_book_corners_on_frame(img2, mapped_corners)

plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB))
plt.title("Mapped Book Corners in Video Frame")
plt.axis("off")
plt.show()
